In [1]:
import os
import pandas as pd
import numpy as np
import time

#### Below we are reading in our dictionary Excel file specifying the sheet name and the columns (C:C, E:E) that we care about. 


In [2]:
glos = pd.read_excel('CollegeScorecardDataDictionary.xlsx',sheet_name='data_dictionary',usecols='C,E', index_col = 1)

#### The [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html) for `index_col` shows that it's 0-indexed which means we're specifying the 2nd of our two columns that we explicitly read in.  Below you can see that E:E *'VARIABLE NAME'* is our index. 

In [3]:
glos.head()

,developer-friendly name
VARIABLE NAME,
UNITID,id
OPEID,ope8_id
OPEID6,ope6_id
INSTNM,name
CITY,city


#### Next we specify the sub-string we're looking for in our column headers and convert the DataFrame to a dictionary. 

In [4]:
lf = 'male.completed_by.4'
glos = glos.to_dict()

In [5]:
default_cols = ['INSTNM']
cols = default_cols + [k for k,v in glos['developer-friendly name'].items() if lf in str(v)]

#### Above we set the columns we want to load from our annual data files with a [list comprehension](http://www.secnetix.de/olli/Python/list_comprehensions.hawk). We are only pulling in institution name and male/female 4 year graduation rate. We could simply remove the '4' from `lf` and we'd have all graduation rate data.

In [6]:
[k for k,v in glos['developer-friendly name'].items() if 'male.completed_by.' in str(v)]

['FEMALE_COMP_ORIG_YR6_RT',
 'MALE_COMP_ORIG_YR3_RT',
 'MALE_COMP_ORIG_YR2_RT',
 'FEMALE_COMP_ORIG_YR3_RT',
 'FEMALE_COMP_ORIG_YR8_RT',
 'MALE_COMP_ORIG_YR4_RT',
 'FEMALE_COMP_ORIG_YR2_RT',
 'MALE_COMP_ORIG_YR8_RT',
 'FEMALE_COMP_ORIG_YR4_RT',
 'MALE_COMP_ORIG_YR6_RT']

#### We grab the files in our current directory and begin to iterate over them. Our `usecols` argument is only loading what we need rather than the 2000+ columns that exist. We definitely want to do this as it is faster and will not use RAM needlessly.  

#### `dropna()` is operating on rows `axis=0` as opposed to columns `axis=1` and it is dropping rows where *any* data is missing. If the school provides male rate but not female, we're not keeping that school.

#### `df['year'] = file[6:10]` is adding the year portion of the file name as a new column so that we can later group by each years data.

In [7]:
files = [x for x in os.listdir() if x.endswith('.csv')]

frames = []
for file in files:
    df = pd.read_csv(file,usecols=cols).dropna(axis=0,how='any')
    df['year'] = file[6:10]
    frames.append(df)
df = pd.concat(frames)

#### As you may have gathered the `glos` variable is somewhat contrived. I really only showed it here to give an example of a few of pandas useful features.

#### We rename and we do it in place below. Notice we are not assigning this operation to a `df` variable

In [8]:
df.rename(columns=glos['developer-friendly name'],inplace=True)

#### Some schools did not publish this data so below we are removing schools that supressed this for either males, females, or both.

In [9]:
df = df[(df.values != 'PrivacySuppressed').all(axis=1)]

#### Next we define a new column into existence by simply subtracting one column from another. We perform the operation `astype` float because they were not read in as decimals.

In [10]:
df['delta'] = df['title_iv.female.completed_by.4yrs'].astype(float) - df['title_iv.male.completed_by.4yrs'].astype(float)

#### Since we want to group by this new column our bands must be set. I have chosen greater than 0, less than 0, and exactly 0.

In [11]:
conditions = [(df['delta'] == 0),(df['delta'] > 0),(df['delta'] < 0)]
choices = ['No Difference','Female Higher','Male Higher']

#### Again, we define a column into existence by performing an assignment by way of a [numpy.select](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.select.html) operation.  Now we need only groupby our new column and display the size of the groups!

In [12]:
df['difference'] = np.select(conditions, choices)

df.groupby('difference').size()


difference
Female Higher    39874
Male Higher      11661
No Difference       36
dtype: int64

#### Below we are grouping by the year then getting the average of the `delta` and multiplying by 100 to show as percentage.

In [13]:
df.groupby('year')['delta'].mean()*100

year
2002    1.431179
2003    2.719079
2004    3.487616
2005    4.126044
2006    4.060755
2007    4.114775
2008    4.464936
2009    4.455689
2010    4.010999
2011    4.065882
2012    4.286755
2013    4.421217
2014    4.151333
Name: delta, dtype: float64